In [3]:
# Simulation - Book all available suites for 17/12/2025, 13:00-16:00 (3 hours)
import requests
import json
from datetime import datetime
import os
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv()

# Mews API Configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken', 'E0D439EE522F44368DC78E1BFB03710C-D24FB11DBE31D4621C4817E028D9E1D')
ACCESS_TOKEN = os.getenv('AccessToken', 'C618020DC2C24A6DAEF7B38A012C43C1-246E13D8FC45C4A80E61D412555A97E')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service ID for JOURNEE
JOURNEE_SERVICE_ID = "86fcc6a7-75ce-457a-a425-b3850108b6bf"

# Rate ID for JOURNEE (base weekday rate)
JOURNEE_RATE_ID = "c3c2109d-984a-4ad4-978e-b3850108b8ad"

# Age category for adults in JOURNEE service (IMPORTANT: Different from NUITEE!)
ADULT_AGE_CATEGORY_ID = "a78b7aca-fa0b-4199-8b4e-b3850108b8a5"

# Booking details
# 17/12/2025 13:00 - 16:00 (3 hours) in UTC
START_UTC = "2025-12-17T13:00:00Z"
END_UTC = "2025-12-17T16:00:00Z"

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

# Step 1: Fetch all suites for JOURNEE service
print("="*80)
print("Step 1: Fetching all suites for JOURNEE service")
print("="*80)

suites_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": [JOURNEE_SERVICE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

suites_result = make_mews_request("resourceCategories/getAll", suites_payload)

if not suites_result or "ResourceCategories" not in suites_result:
    print("❌ Failed to fetch suites")
    exit()

# Filter to only show suites and rooms (not buildings/floors/private spaces)
all_suites = [cat for cat in suites_result["ResourceCategories"]
              if cat.get("Type") in ["Suite", "Room"] and cat.get("IsActive")]

print(f"✅ Found {len(all_suites)} suites/rooms for JOURNEE service:")
for suite in all_suites:
    suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
    suite_type = suite.get("Type")
    print(f"   - {suite_name} ({suite_type}): {suite['Id']}")

SUITE_IDS = [suite["Id"] for suite in all_suites]
print()

# Step 2: Create customer with unique timestamp
print("="*80)
print("Step 2: Creating customer with unique timestamp")
print("="*80)

# Generate unique timestamp
timestamp = int(time.time())
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

customer_payload = {
    "Client": "Intense Experience Simulation",
    "FirstName": f"Simulation",
    "LastName": f"User_{timestamp_str}",
    "Email": f"simulation_{timestamp}@intense-experience.test",
    "Phone": f"+32{timestamp % 1000000000}",  # Use last 9 digits of timestamp for phone
    "OverwriteExisting": True
}

print(f"Creating customer:")
print(f"  Name: {customer_payload['FirstName']} {customer_payload['LastName']}")
print(f"  Email: {customer_payload['Email']}")
print(f"  Phone: {customer_payload['Phone']}")
print()

customer_result = make_mews_request("customers/add", customer_payload)

if not customer_result or 'Id' not in customer_result:
    print("❌ Failed to create customer")
    print(json.dumps(customer_result, indent=2))
else:
    customer_id = customer_result['Id']
    print(f"✅ Customer created successfully")
    print(f"   Customer ID: {customer_id}")
    print(f"   Name: {customer_result.get('FirstName')} {customer_result.get('LastName')}")
    print(f"   Email: {customer_result.get('Email')}")
    print()
    
    # Step 3: Create reservations for all suites
    print("="*80)
    print("Step 3: Creating reservations for all suites")
    print(f"Date: 17/12/2025")
    print(f"Time: 13:00 - 16:00 (3 hours)")
    print(f"Service: JOURNEE")
    print("="*80)
    print()
    
    reservations_created = []
    reservations_failed = []
    
    # Create a lookup dictionary for suite names
    suite_names = {suite["Id"]: suite.get("Names", {}).get("fr-FR", "Unknown") for suite in all_suites}
    
    for suite_id in SUITE_IDS:
        suite_name = suite_names.get(suite_id, suite_id)
        print(f"Creating reservation for {suite_name}...")
        
        # Create unique identifier for this reservation
        suite_code = suite_name.replace(' ', '-').replace('Suite', 'S').replace('Chambre', 'C')
        
        reservation_data = {
            "Identifier": f"SIM-{timestamp_str}-{suite_code}",
            "State": "Confirmed",
            "StartUtc": START_UTC,
            "EndUtc": END_UTC,
            "CustomerId": customer_id,
            "BookerId": customer_id,
            "PersonCounts": [
                {
                    "AgeCategoryId": ADULT_AGE_CATEGORY_ID,
                    "Count": 2
                }
            ],
            "RequestedCategoryId": suite_id,
            "RateId": JOURNEE_RATE_ID
        }
        
        reservation_payload = {
            "Client": "Intense Experience Simulation",
            "ServiceId": JOURNEE_SERVICE_ID,
            "Reservations": [reservation_data]
        }
        
        reservation_result = make_mews_request("reservations/add", reservation_payload)
        
        if reservation_result and "Reservations" in reservation_result and reservation_result["Reservations"]:
            reservation = reservation_result["Reservations"][0]
            reservations_created.append({
                "suite": suite_name,
                "reservation_id": reservation["Reservation"]["Id"],
                "number": reservation["Reservation"]["Number"]
            })
            print(f"   ✅ {suite_name}: Reservation #{reservation['Reservation']['Number']} created")
            print(f"      ID: {reservation['Reservation']['Id']}")
        else:
            reservations_failed.append({
                "suite": suite_name,
                "error": reservation_result
            })
            print(f"   ❌ {suite_name}: Failed to create reservation")
            if reservation_result:
                print(f"      Error: {json.dumps(reservation_result, indent=6)}")
        
        print()
    
    # Step 4: Summary
    print("="*80)
    print("SUMMARY")
    print("="*80)
    print(f"Customer: {customer_payload['FirstName']} {customer_payload['LastName']}")
    print(f"  ID: {customer_id}")
    print(f"  Email: {customer_payload['Email']}")
    print(f"  Phone: {customer_payload['Phone']}")
    print()
    print(f"Date: 17/12/2025 (Wednesday)")
    print(f"Time: 13:00 - 16:00 (3 hours)")
    print(f"Service: JOURNEE")
    print()
    print(f"✅ Reservations created: {len(reservations_created)}")
    for res in reservations_created:
        print(f"   - {res['suite']}: Reservation #{res['number']} (ID: {res['reservation_id']})")
    
    if reservations_failed:
        print()
        print(f"❌ Reservations failed: {len(reservations_failed)}")
        for res in reservations_failed:
            print(f"   - {res['suite']}")
    
    print("="*80)
    print(f"\n🎯 Simulation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🔑 Timestamp identifier: {timestamp_str}")


Step 1: Fetching all suites for JOURNEE service
✅ Found 7 suites/rooms for JOURNEE service:
   - Suite GAIA (Suite): f723bd5a-04fe-479c-bab4-b3850108c66f
   - Chambre EUPHORYA (Room): f5539e51-9db0-4082-b87e-b3850108c66f
   - Suite EXTASE (Suite): 68b87fe5-7b78-4067-b5e7-b3850108c66f
   - Chambre IGNIS (Room): 78a614b1-199d-4608-ab89-b3850108c66f
   - Suite INTENSE (Suite): e4706d3a-2a06-4cb7-a449-b3850108c66f
   - Chambre KAIROS (Room): 67ece5a2-65e2-43c5-9079-b3850108c66f
   - Chambre AETHER (Room): 1113bcbe-ad5f-49c7-8dc0-b3850108c66f

Step 2: Creating customer with unique timestamp
Creating customer:
  Name: Simulation User_20251116_163539
  Email: simulation_1763307339@intense-experience.test
  Phone: +32763307339

✅ Customer created successfully
   Customer ID: 48714353-5cb4-4a8b-8c6a-b3970100fd0b
   Name: Simulation User_20251116_163539
   Email: simulation_1763307339@intense-experience.test

Step 3: Creating reservations for all suites
Date: 17/12/2025
Time: 13:00 - 16:00 (3 ho

In [3]:

# Simulation - Book all available suites for 18/12/2025, 10:00-16:00 (6 hours)
import requests
import json
from datetime import datetime
import os
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv()

# Mews API Configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken', 'E0D439EE522F44368DC78E1BFB03710C-D24FB11DBE31D4621C4817E028D9E1D')
ACCESS_TOKEN = os.getenv('AccessToken', 'C618020DC2C24A6DAEF7B38A012C43C1-246E13D8FC45C4A80E61D412555A97E')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service ID for JOURNEE
JOURNEE_SERVICE_ID = "86fcc6a7-75ce-457a-a425-b3850108b6bf"

# Rate ID for JOURNEE (base weekday rate)
JOURNEE_RATE_ID = "c3c2109d-984a-4ad4-978e-b3850108b8ad"

# Age category for adults in JOURNEE service (IMPORTANT: Different from NUITEE!)
ADULT_AGE_CATEGORY_ID = "a78b7aca-fa0b-4199-8b4e-b3850108b8a5"

# Booking details
# 18/12/2025 10:00 - 16:00 (6 hours) in UTC
START_UTC = "2025-12-18T10:00:00Z"
END_UTC = "2025-12-18T16:00:00Z"

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

# Step 1: Fetch all suites for JOURNEE service
print("="*80)
print("Step 1: Fetching all suites for JOURNEE service")
print("="*80)

suites_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": [JOURNEE_SERVICE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

suites_result = make_mews_request("resourceCategories/getAll", suites_payload)

if not suites_result or "ResourceCategories" not in suites_result:
    print("❌ Failed to fetch suites")
    exit()

# Filter to only show suites and rooms (not buildings/floors/private spaces)
all_suites = [cat for cat in suites_result["ResourceCategories"]
              if cat.get("Type") in ["Suite", "Room"] and cat.get("IsActive")]

print(f"✅ Found {len(all_suites)} suites/rooms for JOURNEE service:")
for suite in all_suites:
    suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
    suite_type = suite.get("Type")
    print(f"   - {suite_name} ({suite_type}): {suite['Id']}")

SUITE_IDS = [suite["Id"] for suite in all_suites]
print()

# Step 2: Create customer with unique timestamp
print("="*80)
print("Step 2: Creating customer with unique timestamp")
print("="*80)

# Generate unique timestamp
timestamp = int(time.time())
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

customer_payload = {
    "Client": "Intense Experience Simulation",
    "FirstName": f"Simulation",
    "LastName": f"User_{timestamp_str}",
    "Email": f"simulation_{timestamp}@intense-experience.test",
    "Phone": f"+32{timestamp % 1000000000}",  # Use last 9 digits of timestamp for phone
    "OverwriteExisting": True
}

print(f"Creating customer:")
print(f"  Name: {customer_payload['FirstName']} {customer_payload['LastName']}")
print(f"  Email: {customer_payload['Email']}")
print(f"  Phone: {customer_payload['Phone']}")
print()

customer_result = make_mews_request("customers/add", customer_payload)

if not customer_result or 'Id' not in customer_result:
    print("❌ Failed to create customer")
    print(json.dumps(customer_result, indent=2))
else:
    customer_id = customer_result['Id']
    print(f"✅ Customer created successfully")
    print(f"   Customer ID: {customer_id}")
    print(f"   Name: {customer_result.get('FirstName')} {customer_result.get('LastName')}")
    print(f"   Email: {customer_result.get('Email')}")
    print()
    
    # Step 3: Create reservations for all suites
    print("="*80)
    print("Step 3: Creating reservations for all suites")
    print(f"Date: 18/12/2025")
    print(f"Time: 10:00 - 16:00 (6 hours)")
    print(f"Service: JOURNEE")
    print("="*80)
    print()
    
    reservations_created = []
    reservations_failed = []
    
    # Create a lookup dictionary for suite names
    suite_names = {suite["Id"]: suite.get("Names", {}).get("fr-FR", "Unknown") for suite in all_suites}
    
    for suite_id in SUITE_IDS:
        suite_name = suite_names.get(suite_id, suite_id)
        print(f"Creating reservation for {suite_name}...")
        
        # Create unique identifier for this reservation
        suite_code = suite_name.replace(' ', '-').replace('Suite', 'S').replace('Chambre', 'C')
        
        reservation_data = {
            "Identifier": f"SIM-{timestamp_str}-{suite_code}",
            "State": "Confirmed",
            "StartUtc": START_UTC,
            "EndUtc": END_UTC,
            "CustomerId": customer_id,
            "BookerId": customer_id,
            "PersonCounts": [
                {
                    "AgeCategoryId": ADULT_AGE_CATEGORY_ID,
                    "Count": 2
                }
            ],
            "RequestedCategoryId": suite_id,
            "RateId": JOURNEE_RATE_ID
        }
        
        reservation_payload = {
            "Client": "Intense Experience Simulation",
            "ServiceId": JOURNEE_SERVICE_ID,
            "Reservations": [reservation_data]
        }
        
        reservation_result = make_mews_request("reservations/add", reservation_payload)
        
        if reservation_result and "Reservations" in reservation_result and reservation_result["Reservations"]:
            reservation = reservation_result["Reservations"][0]
            reservations_created.append({
                "suite": suite_name,
                "reservation_id": reservation["Reservation"]["Id"],
                "number": reservation["Reservation"]["Number"]
            })
            print(f"   ✅ {suite_name}: Reservation #{reservation['Reservation']['Number']} created")
            print(f"      ID: {reservation['Reservation']['Id']}")
        else:
            reservations_failed.append({
                "suite": suite_name,
                "error": reservation_result
            })
            print(f"   ❌ {suite_name}: Failed to create reservation")
            if reservation_result:
                print(f"      Error: {json.dumps(reservation_result, indent=6)}")
        
        print()
    
    # Step 4: Summary
    print("="*80)
    print("SUMMARY")
    print("="*80)
    print(f"Customer: {customer_payload['FirstName']} {customer_payload['LastName']}")
    print(f"  ID: {customer_id}")
    print(f"  Email: {customer_payload['Email']}")
    print(f"  Phone: {customer_payload['Phone']}")
    print()
    print(f"Date: 18/12/2025 (Thursday)")
    print(f"Time: 10:00 - 16:00 (6 hours)")
    print(f"Service: JOURNEE")
    print()
    print(f"✅ Reservations created: {len(reservations_created)}")
    for res in reservations_created:
        print(f"   - {res['suite']}: Reservation #{res['number']} (ID: {res['reservation_id']})")
    
    if reservations_failed:
        print()
        print(f"❌ Reservations failed: {len(reservations_failed)}")
        for res in reservations_failed:
            print(f"   - {res['suite']}")
    
    print("="*80)
    print(f"\n🎯 Simulation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🔑 Timestamp identifier: {timestamp_str}")


Step 1: Fetching all suites for JOURNEE service
✅ Found 7 suites/rooms for JOURNEE service:
   - Suite GAIA (Suite): f723bd5a-04fe-479c-bab4-b3850108c66f
   - Chambre EUPHORYA (Room): f5539e51-9db0-4082-b87e-b3850108c66f
   - Suite EXTASE (Suite): 68b87fe5-7b78-4067-b5e7-b3850108c66f
   - Chambre IGNIS (Room): 78a614b1-199d-4608-ab89-b3850108c66f
   - Suite INTENSE (Suite): e4706d3a-2a06-4cb7-a449-b3850108c66f
   - Chambre KAIROS (Room): 67ece5a2-65e2-43c5-9079-b3850108c66f
   - Chambre AETHER (Room): 1113bcbe-ad5f-49c7-8dc0-b3850108c66f

Step 2: Creating customer with unique timestamp
Creating customer:
  Name: Simulation User_20251122_114837
  Email: simulation_1763808517@intense-experience.test
  Phone: +32763808517

✅ Customer created successfully
   Customer ID: d120dccd-e2a2-467c-8ebc-b39d00b22744
   Name: Simulation User_20251122_114837
   Email: simulation_1763808517@intense-experience.test

Step 3: Creating reservations for all suites
Date: 18/12/2025
Time: 10:00 - 16:00 (6 ho

In [ ]:
# Check availability for December 16th and 17th, 2025 (JOURNEE service)
import requests
import json
from datetime import datetime, timedelta, timezone
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Mews API Configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken', 'E0D439EE522F44368DC78E1BFB03710C-D24FB11DBE31D4621C4817E028D9E1D')
ACCESS_TOKEN = os.getenv('AccessToken', 'C618020DC2C24A6DAEF7B38A012C43C1-246E13D8FC45C4A80E61D412555A97E')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service ID for JOURNEE
JOURNEE_SERVICE_ID = "86fcc6a7-75ce-457a-a425-b3850108b6bf"

# Booking constraints
DAY_MIN_HOURS = 3
DAY_MAX_HOURS = 6
CLEANING_BUFFER_HOURS = 1
ARRIVAL_TIMES = ['13:00', '14:00', '15:00', '16:00', '17:00']
DEPARTURE_TIMES = ['14:00', '15:00', '16:00', '17:00', '18:00']

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

print("="*100)
print("AVAILABILITY CHECK FOR DECEMBER 16-17, 2025 (JOURNEE SERVICE)")
print("="*100)
print()

# Step 1: Fetch all suites for JOURNEE service
print("Step 1: Fetching all suites for JOURNEE service")
print("-"*100)

suites_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": [JOURNEE_SERVICE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

suites_result = make_mews_request("resourceCategories/getAll", suites_payload)

if not suites_result or "ResourceCategories" not in suites_result:
    print("❌ Failed to fetch suites")
    exit()

all_suites = [cat for cat in suites_result["ResourceCategories"]
              if cat.get("Type") in ["Suite", "Room"] and cat.get("IsActive")]
suite_ids = [suite["Id"] for suite in all_suites]

print(f"✅ Found {len(all_suites)} active suites/rooms:")
for suite in all_suites:
    suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
    suite_type = suite.get("Type")
    suite_id_short = suite['Id'][:8]
    print(f"   - {suite_name} ({suite_type}): {suite_id_short}...")

print()

# Step 2: Check availability for December 16-17
dates_to_check = [
    datetime(2025, 12, 16, 0, 0, 0, tzinfo=timezone.utc),
    datetime(2025, 12, 17, 0, 0, 0, tzinfo=timezone.utc)
]

print("Step 2: Fetching reservations for December 16-17, 2025")
print("-"*100)

# Fetch reservations for the date range (with buffer)
start_date = dates_to_check[0]
end_date = dates_to_check[-1] + timedelta(days=1)
buffered_end = end_date + timedelta(hours=CLEANING_BUFFER_HOURS)

print(f"Query range: {start_date.strftime('%Y-%m-%d')} to {buffered_end.strftime('%Y-%m-%d %H:%M')} (with buffer)")
print()

reservations_payload = {
    "Client": "Intense Experience Availability Check",
    "StartUtc": start_date.isoformat(),
    "EndUtc": buffered_end.isoformat()
}

reservations_result = make_mews_request("reservations/getAll", reservations_payload)

if not reservations_result:
    print("❌ Failed to fetch reservations")
    exit()

reservations = reservations_result.get("Reservations", [])
print(f"✅ Found {len(reservations)} total reservations in date range")
print()

# Filter reservations by suite
suite_reservations = {}
for suite_id in suite_ids:
    suite_reservations[suite_id] = []
    for res in reservations:
        if res.get('RequestedCategoryId') == suite_id:
            suite_reservations[suite_id].append(res)

print("Reservations by suite:")
for suite in all_suites:
    suite_id = suite['Id']
    suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
    res_count = len(suite_reservations[suite_id])
    print(f"   - {suite_name}: {res_count} reservation(s)")
    for res in suite_reservations[suite_id]:
        res_start = datetime.fromisoformat(res.get('StartUtc', '').replace('Z', '+00:00'))
        res_end = datetime.fromisoformat(res.get('EndUtc', '').replace('Z', '+00:00'))
        res_id_short = res.get('Id', 'unknown')[:8]
        print(f"      • {res_id_short}: {res_start.strftime('%Y-%m-%d %H:%M')} → {res_end.strftime('%H:%M')}")

print()
print("="*100)

# Step 3: Check availability for each date
for check_date in dates_to_check:
    date_str = check_date.strftime('%Y-%m-%d')
    day_name = check_date.strftime('%A')
    
    print()
    print(f"📅 AVAILABILITY ANALYSIS FOR {date_str} ({day_name})")
    print("="*100)
    
    suite_availability = {}
    
    # Check each suite
    for suite in all_suites:
        suite_id = suite['Id']
        suite_name = suite.get("Names", {}).get("fr-FR", "Unknown")
        available_slots = []
        blocked_slots = []
        
        # Check all time slot combinations
        for arrival_time in ARRIVAL_TIMES:
            for departure_time in DEPARTURE_TIMES:
                # Calculate duration
                arr_hour = int(arrival_time.split(':')[0])
                dep_hour = int(departure_time.split(':')[0])
                duration = dep_hour - arr_hour
                
                # Check if duration is within limits
                if duration < DAY_MIN_HOURS or duration > DAY_MAX_HOURS:
                    continue
                
                # Create datetime objects for this slot
                slot_start = datetime.combine(check_date.date(), datetime.strptime(arrival_time, '%H:%M').time()).replace(tzinfo=timezone.utc)
                slot_end = datetime.combine(check_date.date(), datetime.strptime(departure_time, '%H:%M').time()).replace(tzinfo=timezone.utc)
                slot_end_buffered = slot_end + timedelta(hours=CLEANING_BUFFER_HOURS)
                
                # Check if this slot conflicts with any reservation
                is_available = True
                conflicting_reservation = None
                
                for reservation in suite_reservations[suite_id]:
                    res_start = datetime.fromisoformat(reservation.get('StartUtc', '').replace('Z', '+00:00'))
                    res_end = datetime.fromisoformat(reservation.get('EndUtc', '').replace('Z', '+00:00'))
                    
                    # Check for overlap (including buffer)
                    if not (slot_end_buffered <= res_start or slot_start >= res_end):
                        is_available = False
                        conflicting_reservation = reservation
                        break
                
                if is_available:
                    available_slots.append({
                        'arrival': arrival_time,
                        'departure': departure_time,
                        'duration': duration
                    })
                else:
                    blocked_slots.append({
                        'arrival': arrival_time,
                        'departure': departure_time,
                        'duration': duration,
                        'conflict': conflicting_reservation
                    })
        
        suite_availability[suite_id] = {
            'name': suite_name,
            'available': available_slots,
            'blocked': blocked_slots
        }
    
    # Summary for this date
    total_available_slots = sum(len(data['available']) for data in suite_availability.values())
    has_availability = total_available_slots > 0
    
    print(f"\n🎯 SUMMARY:")
    print(f"   Total suites checked: {len(all_suites)}")
    print(f"   Total available time slots: {total_available_slots}")
    print(f"   Status: {'✅ AVAILABLE' if has_availability else '❌ FULLY BOOKED'}")
    print()
    
    # Detailed breakdown by suite
    print(f"📊 SUITE-BY-SUITE BREAKDOWN:")
    print()
    
    for suite in all_suites:
        suite_id = suite['Id']
        data = suite_availability[suite_id]
        suite_name = data['name']
        num_available = len(data['available'])
        num_blocked = len(data['blocked'])
        
        if num_available > 0:
            print(f"✅ {suite_name}: {num_available} available slot(s)")
            print(f"   Available time slots:")
            for slot in data['available']:
                print(f"      • {slot['arrival']} → {slot['departure']} ({slot['duration']}h)")
        else:
            print(f"❌ {suite_name}: NO available slots (fully booked)")
        
        if num_blocked > 0:
            print(f"   Blocked time slots: {num_blocked}")
            # Show first 3 blocked slots with details
            for i, slot in enumerate(data['blocked'][:3]):
                conflict = slot['conflict']
                res_id_short = conflict.get('Id', 'unknown')[:8]
                res_start = datetime.fromisoformat(conflict.get('StartUtc', '').replace('Z', '+00:00'))
                res_end = datetime.fromisoformat(conflict.get('EndUtc', '').replace('Z', '+00:00'))
                print(f"      ✗ {slot['arrival']} → {slot['departure']} blocked by reservation {res_id_short}")
                print(f"         Conflicting reservation: {res_start.strftime('%Y-%m-%d %H:%M')} → {res_end.strftime('%H:%M')}")
            
            if num_blocked > 3:
                print(f"      ... and {num_blocked - 3} more blocked slots")
        
        print()
    
    print("-"*100)

print()
print("="*100)
print("✅ AVAILABILITY CHECK COMPLETE")
print("="*100)


AVAILABILITY CHECK FOR DECEMBER 16-17, 2025 (JOURNEE SERVICE)

Step 1: Fetching all suites for JOURNEE service
----------------------------------------------------------------------------------------------------
✅ Found 7 active suites/rooms:
   - Suite GAIA (Suite): f723bd5a...
   - Chambre EUPHORYA (Room): f5539e51...
   - Suite EXTASE (Suite): 68b87fe5...
   - Chambre IGNIS (Room): 78a614b1...
   - Suite INTENSE (Suite): e4706d3a...
   - Chambre KAIROS (Room): 67ece5a2...
   - Chambre AETHER (Room): 1113bcbe...

Step 2: Fetching reservations for December 16-17, 2025
----------------------------------------------------------------------------------------------------
Query range: 2025-12-16 to 2025-12-18 01:00 (with buffer)

✅ Found 8 total reservations in date range

Reservations by suite:
   - Suite GAIA: 1 reservation(s)
      • e309ae21: 2025-12-17 13:00 → 16:00
   - Chambre EUPHORYA: 1 reservation(s)
      • 3796c3a7: 2025-12-17 13:00 → 16:00
   - Suite EXTASE: 1 reservation(s)
 

In [1]:
# Check availability for December 5th, 2025 (JOURNEE service) - Suite e4706d3a only
import requests
import json
from datetime import datetime, timedelta, timezone
import os
from dotenv import load_dotenv
import pytz

# Load environment variables
load_dotenv()

# Mews API Configuration
MEWS_BASE_URL = "https://api.mews-demo.com/api/connector/v1"
CLIENT_TOKEN = os.getenv('ClientToken', 'E0D439EE522F44368DC78E1BFB03710C-D24FB11DBE31D4621C4817E028D9E1D')
ACCESS_TOKEN = os.getenv('AccessToken', 'C618020DC2C24A6DAEF7B38A012C43C1-246E13D8FC45C4A80E61D412555A97E')
ENTERPRISE_ID = "c390a691-e9a0-4aa0-860c-b3850108ab4c"

# Service ID for JOURNEE
JOURNEE_SERVICE_ID = "86fcc6a7-75ce-457a-a425-b3850108b6bf"

# Target suite ID
TARGET_SUITE_ID = "e4706d3a-2a06-4cb7-a449-b3850108c66f"

# Booking constraints
DAY_MIN_HOURS = 3
DAY_MAX_HOURS = 6
CLEANING_BUFFER_HOURS = 1
ARRIVAL_TIMES = ['09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00']
DEPARTURE_TIMES = ['10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00']

# Brussels timezone
BRUSSELS_TZ = pytz.timezone('Europe/Brussels')

def make_mews_request(endpoint, payload):
    """Make a request to Mews API"""
    url = f"{MEWS_BASE_URL}/{endpoint}"
    payload.update({
        "ClientToken": CLIENT_TOKEN,
        "AccessToken": ACCESS_TOKEN
    })
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return None

print("="*100)
print("AVAILABILITY CHECK FOR DECEMBER 5, 2025 (JOURNEE SERVICE) - SUITE e4706d3a ONLY")
print("="*100)
print()

# Step 1: Fetch suite information
print("Step 1: Fetching suite information")
print("-"*100)

suites_payload = {
    "EnterpriseIds": [ENTERPRISE_ID],
    "ServiceIds": [JOURNEE_SERVICE_ID],
    "IncludeDefault": False,
    "Limitation": {"Count": 100}
}

suites_result = make_mews_request("resourceCategories/getAll", suites_payload)

if not suites_result or "ResourceCategories" not in suites_result:
    print("❌ Failed to fetch suites")
    exit()

# Find the target suite
target_suite = None
for cat in suites_result["ResourceCategories"]:
    if cat.get("Id") == TARGET_SUITE_ID and cat.get("IsActive"):
        target_suite = cat
        break

if not target_suite:
    print(f"❌ Suite {TARGET_SUITE_ID} not found or not active")
    exit()

suite_name = target_suite.get("Names", {}).get("fr-FR", "Unknown")
suite_type = target_suite.get("Type")
print(f"✅ Found target suite: {suite_name} ({suite_type})")
print(f"   Suite ID: {TARGET_SUITE_ID}")
print()

# Step 2: Check availability for December 5
# Create date in Brussels timezone
check_date = BRUSSELS_TZ.localize(datetime(2025, 12, 5, 0, 0, 0))
date_str = check_date.strftime('%Y-%m-%d')
day_name = check_date.strftime('%A')

print("Step 2: Fetching reservations for December 5, 2025")
print("-"*100)

# Fetch reservations for the date (with buffer)
start_date = check_date
end_date = check_date + timedelta(days=1)
buffered_end = end_date + timedelta(hours=CLEANING_BUFFER_HOURS)

# Convert to UTC for API request
start_date_utc = start_date.astimezone(timezone.utc)
buffered_end_utc = buffered_end.astimezone(timezone.utc)

print(f"Query range (Brussels time): {start_date.strftime('%Y-%m-%d %H:%M %Z')} to {buffered_end.strftime('%Y-%m-%d %H:%M %Z')} (with buffer)")
print(f"Query range (UTC): {start_date_utc.strftime('%Y-%m-%d %H:%M %Z')} to {buffered_end_utc.strftime('%Y-%m-%d %H:%M %Z')}")
print()

reservations_payload = {
    "Client": "Intense Experience Availability Check",
    "StartUtc": start_date_utc.isoformat(),
    "EndUtc": buffered_end_utc.isoformat()
}

reservations_result = make_mews_request("reservations/getAll", reservations_payload)

if not reservations_result:
    print("❌ Failed to fetch reservations")
    exit()

reservations = reservations_result.get("Reservations", [])
print(f"✅ Found {len(reservations)} total reservations in date range")
print()

# Filter reservations for target suite
suite_reservations = [res for res in reservations if res.get('RequestedCategoryId') == TARGET_SUITE_ID]

print(f"Reservations for {suite_name}:")
if len(suite_reservations) == 0:
    print("   No reservations found")
else:
    for res in suite_reservations:
        res_start_utc = datetime.fromisoformat(res.get('StartUtc', '').replace('Z', '+00:00'))
        res_end_utc = datetime.fromisoformat(res.get('EndUtc', '').replace('Z', '+00:00'))
        # Convert to Brussels time for display
        res_start = res_start_utc.astimezone(BRUSSELS_TZ)
        res_end = res_end_utc.astimezone(BRUSSELS_TZ)
        res_id_short = res.get('Id', 'unknown')[:8]
        print(f"   • {res_id_short}: {res_start.strftime('%Y-%m-%d %H:%M %Z')} → {res_end.strftime('%H:%M %Z')}")

print()
print("="*100)

# Step 3: Check availability for December 5
print()
print(f"📅 AVAILABILITY ANALYSIS FOR {date_str} ({day_name}) - Brussels Time")
print("="*100)

available_slots = []
blocked_slots = []

# Check all time slot combinations
for arrival_time in ARRIVAL_TIMES:
    for departure_time in DEPARTURE_TIMES:
        # Calculate duration
        arr_hour = int(arrival_time.split(':')[0])
        dep_hour = int(departure_time.split(':')[0])
        duration = dep_hour - arr_hour
        
        # Check if duration is within limits
        if duration < DAY_MIN_HOURS or duration > DAY_MAX_HOURS:
            continue
        
        # Create datetime objects for this slot in Brussels timezone
        slot_start = BRUSSELS_TZ.localize(datetime.combine(check_date.date(), datetime.strptime(arrival_time, '%H:%M').time()))
        slot_end = BRUSSELS_TZ.localize(datetime.combine(check_date.date(), datetime.strptime(departure_time, '%H:%M').time()))
        slot_end_buffered = slot_end + timedelta(hours=CLEANING_BUFFER_HOURS)
        
        # Check if this slot conflicts with any reservation
        is_available = True
        conflicting_reservation = None
        
        for reservation in suite_reservations:
            res_start_utc = datetime.fromisoformat(reservation.get('StartUtc', '').replace('Z', '+00:00'))
            res_end_utc = datetime.fromisoformat(reservation.get('EndUtc', '').replace('Z', '+00:00'))
            # Convert to Brussels time for comparison
            res_start = res_start_utc.astimezone(BRUSSELS_TZ)
            res_end = res_end_utc.astimezone(BRUSSELS_TZ)
            
            # Check for overlap (including buffer)
            if not (slot_end_buffered <= res_start or slot_start >= res_end):
                is_available = False
                conflicting_reservation = reservation
                break
        
        if is_available:
            available_slots.append({
                'arrival': arrival_time,
                'departure': departure_time,
                'duration': duration
            })
        else:
            blocked_slots.append({
                'arrival': arrival_time,
                'departure': departure_time,
                'duration': duration,
                'conflict': conflicting_reservation
            })

# Summary
total_available_slots = len(available_slots)
has_availability = total_available_slots > 0

print(f"\n🎯 SUMMARY:")
print(f"   Suite: {suite_name}")
print(f"   Total available time slots: {total_available_slots}")
print(f"   Status: {'✅ AVAILABLE' if has_availability else '❌ FULLY BOOKED'}")
print()

# Detailed breakdown
print(f"📊 DETAILED BREAKDOWN:")
print()

if total_available_slots > 0:
    print(f"✅ {suite_name}: {total_available_slots} available slot(s)")
    print(f"   Available time slots:")
    for slot in available_slots:
        print(f"      • {slot['arrival']} → {slot['departure']} ({slot['duration']}h)")
else:
    print(f"❌ {suite_name}: NO available slots (fully booked)")

if len(blocked_slots) > 0:
    print(f"\n   Blocked time slots: {len(blocked_slots)}")
    # Show first 3 blocked slots with details
    for i, slot in enumerate(blocked_slots[:3]):
        conflict = slot['conflict']
        res_id_short = conflict.get('Id', 'unknown')[:8]
        res_start_utc = datetime.fromisoformat(conflict.get('StartUtc', '').replace('Z', '+00:00'))
        res_end_utc = datetime.fromisoformat(conflict.get('EndUtc', '').replace('Z', '+00:00'))
        # Convert to Brussels time for display
        res_start = res_start_utc.astimezone(BRUSSELS_TZ)
        res_end = res_end_utc.astimezone(BRUSSELS_TZ)
        print(f"      ✗ {slot['arrival']} → {slot['departure']} blocked by reservation {res_id_short}")
        print(f"         Conflicting reservation: {res_start.strftime('%Y-%m-%d %H:%M %Z')} → {res_end.strftime('%H:%M %Z')}")
    
    if len(blocked_slots) > 3:
        print(f"      ... and {len(blocked_slots) - 3} more blocked slots")

print()
print("-"*100)

print()
print("="*100)
print("✅ AVAILABILITY CHECK COMPLETE")
print("="*100)


AVAILABILITY CHECK FOR DECEMBER 5, 2025 (JOURNEE SERVICE) - SUITE e4706d3a ONLY

Step 1: Fetching suite information
----------------------------------------------------------------------------------------------------
✅ Found target suite: Suite INTENSE (Suite)
   Suite ID: e4706d3a-2a06-4cb7-a449-b3850108c66f

Step 2: Fetching reservations for December 5, 2025
----------------------------------------------------------------------------------------------------
Query range (Brussels time): 2025-12-05 00:00 CET to 2025-12-06 01:00 CET (with buffer)
Query range (UTC): 2025-12-04 23:00 UTC to 2025-12-06 00:00 UTC

✅ Found 1 total reservations in date range

Reservations for Suite INTENSE:
   • 1beff2f8: 2025-12-05 09:00 CET → 12:00 CET


📅 AVAILABILITY ANALYSIS FOR 2025-12-05 (Friday) - Brussels Time

🎯 SUMMARY:
   Suite: Suite INTENSE
   Total available time slots: 10
   Status: ✅ AVAILABLE

📊 DETAILED BREAKDOWN:

✅ Suite INTENSE: 10 available slot(s)
   Available time slots:
      • 12:00